# Notebook for generating shutterfiles

Setting inputs: length of flight path, t offset and repetition frequency

Inputs for the shutter file generation: number of windows, and, for each window, tof/wavelength start-end and bin size 

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import csv
import sys
from TOF_routines import tof2l, l2tof

%matplotlib inline

In [ ]:
L=56.4 #in meters
t0=0.00 #in ms
f=10 #in Hz
t2overlap=1000/f #in ms

In [ ]:
# number of acquisition windows
n_windows=input("Enter the number of acquisition window: ")



In [ ]:
windows=np.zeros((int(n_windows),2))
windows[0][0]=l2tof(1,t0,L)*1e+3 # in ms
windows[0][1]=l2tof(2.45,t0,L)*1e+3

windows[1][0]=l2tof(2.5,t0,L)*1e+3
windows[1][1]=l2tof(3.3,t0,L)*1e+3

windows[2][0]=l2tof(3.35,t0,L)*1e+3
windows[2][1]=l2tof(4.6,t0,L)*1e+3

windows[3][0]=l2tof(4.65,t0,L)*1e+3
windows[3][1]=l2tof(6,t0,L)*1e+3

print(windows)

#windows[4][0]=22.34
#windows[4][1]=36

In [ ]:
# Windows durations
delta_tof=windows[:,1]-windows[:,0]
print("delta tof:")
print(delta_tof)

In [ ]:
clock_frequency=np.zeros((14))
clock_divider=np.zeros(int(n_windows))
clock_time=np.zeros((14))
for i in range (0,14):
    clock_frequency[i]=100/(2**i)
    clock_time[i]=0.00001*(2**i)
    #clock_divider[i]=i
range_covered=11800*clock_time
print("clock frequency:")
print(clock_frequency)
print("clock time:")
print(clock_time)
#print(clock_divider)
print("range covered:")
print(range_covered)


In [ ]:
for i in range (0,int(n_windows)):
    for j in range (0,14):
            if delta_tof[i]<range_covered[j]:
                clock_divider[i]=j
                break
print("clock divider:")
print(clock_divider)
    

In [ ]:
clock_time[int(clock_divider[2])]

In [ ]:
# Bin width. THIS IS AN INPUT
bin_width=np.zeros(int(n_windows));
        
bin_width[0]=40.96
bin_width[1]=20.48
bin_width[2]=20.48
bin_width[3]=40.96
#bin_width[4]=40.96

print(bin_width)

In [ ]:
# Shutter duration
n_shutter=int(n_windows)-1
duration=np.zeros((n_shutter))
for i in range (0,n_shutter):
    duration[i]=windows[i+1][0]-windows[i][1]
print(duration)

In [ ]:
# Resolution calculation
min_res=bin_width*0.001/windows[:,0]
max_res=bin_width*0.001/windows[:,1]
print(min_res*100)
print(max_res*100)

In [ ]:
# calculation of the number of file and space needed
no_file=np.ceil(delta_tof*1000/bin_width);
tot_file=np.sum(no_file)
print("Number of file produced: ")
print(tot_file)
print("Number of fiels")
print(no_file)

In [ ]:
#base_path = "/path/to/directory/holding/file/"
filename = "data/alpha.txt"
filename2= "data/lambda.txt"
#path_to_file = os.path.join(base_path, filename)
#fd = open(path_to_file , 'r')
fp = open(filename , 'r')
fp2 = open(filename2, 'r')
#spectrum=csv.reader(fp, delimiter="\t")

In [ ]:
#data = np.genfromtxt(filename, delimiter = '\t', skip_header=1)
data = np.genfromtxt(filename)
l = np.genfromtxt(filename2)

In [ ]:
#type(data)
y=data.T
x =l.T

#x_axis=x(0)
#y_axis=y(0)
#np.delete(x,0)
#np.delete(y,0)

In [ ]:

for i in range (0,int(n_windows)):
    win_plot_x_1=(windows[i][0],windows[i][0])
    win_plot_y_1=(min(y),max(y))
    plt.plot(win_plot_x_1,win_plot_y_1,'r-')
    win_plot_x_2=(windows[i][1],windows[i][1])
    win_plot_y_2=(min(y),max(y))
    plt.plot(win_plot_x_2,win_plot_y_2,'r-')
plt.plot(l2tof(x,t0,L)*1e+3,y)
plt.title("Theoretical Iron spectrum with overimposed windows")
plt.xlabel("TOF [ms]")
   


In [ ]:
#write a txt file for the shutter
filename_out = 'FileShutter.txt'

with open(filename_out, 'w') as f:
    for i in range(0,int(n_windows)):
            f.write(str(windows[i][0]))
            f.write("\t")
            f.write(str(windows[i][1]))
            f.write("\t")
            f.write(str(clock_divider[i]))
            f.write("\t")
            f.write(str(bin_width[i]))
            f.write("\n")
